In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import random

In [2]:
request_headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}

url = 'https://www.idealista.com/alquiler-viviendas/barcelona-barcelona/'

response_main = requests.get(url, headers=request_headers, timeout=5)
if response_main.status_code == 200:
    print('Response successful') 
else:
    print(response_main.status_code)

Response successful


In [3]:
soup = BeautifulSoup(response_main.content, "html.parser")

In [7]:
property_links = []

for i in range (1,61):
    url = f'https://www.idealista.com/alquiler-viviendas/barcelona-barcelona/pagina-{i}.htm'
    response_main = requests.get(url, headers=request_headers, timeout=5)
    soup = BeautifulSoup(response_main.content, "html.parser")
    links = soup.find_all(class_="item-link")
    for link in links:
        href = link.get("href")
        if href:
            full_link = "https://www.idealista.com/" + href
            property_links.append(full_link)
print(len(property_links))

1800


In [ ]:
property_details = []

with open("property_data_idealista.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Price_EUR_per_month", "Neighbourhood", "Person", "Number_of_rooms", "Square_meters", "Exterior_interior", "Floor", "Lift", "Garage"])
    for property in property_links:
        response = requests.get(property, headers=request_headers)
        property_data = BeautifulSoup(response.content, "html.parser")
        for apartment in property_data:
            try:
                title = property_data.find(class_="main-info__title-main").get_text()
            except AttributeError:
                title = ""
            try:
                price = float(property_data.find(class_="info-data-price").get_text()[:-5].replace(".", "").replace(",", "."))
            except AttributeError:
                 price = ""
            try:
                neighbourhood = property_data.find(class_="main-info__title-minor").get_text()
            except AttributeError:
                neighbourhood = ""
            try:
                person = property_data.find("div", class_="professional-name").get_text().split()[0].strip()
            except AttributeError:
                person = ""
            features = property_data.find(class_="info-features")
            features_text = property_data.find(class_="info-features").get_text()
            if "hab." in features_text:
                nr_of_rooms = float(next((span.get_text(strip=True).replace(" hab.", "") for span in features if "hab" in span.get_text()), ""))
            else:
                nr_of_rooms = ""
            if "m²" in features_text:
                square_meters = float(next((span.get_text(strip=True).replace(" m²", "") for span in features if "m²" in span.get_text()), ""))
            else:
                square_meters = ""
            if "exterior" in features_text:
                interior_exterior = "exterior"
            elif "interior" in features_text:
                interior_exterior = "interior"
            else:
                interior_exterior = ""
            if "con ascensor" in features_text:
                lift = float(1)
            elif "sin ascensor" in features_text:
                lift = float(0)
            else:
                lift = ""
            if "Planta" in features_text:
                floor_details = (next((span.get_text(strip=True) for span in features if "Planta" in span.get_text()), ""))
                floor_nr = float(floor_details.split()[1].strip("ª"))
            elif "Bajo" in features_text:
                floor_nr = "Bajo"
            elif "Entreplanta" in features_text:
                floor_nr = "Entreplanta"
            else:
                floor_nr = ""
            if "Garaje incluido" in features_text:
                garage = "Garage included"
            elif "Garaje opc." in features_text:
                garage = "Garage optional"
            else:
                garage = ""
        property_details.append([title, price, neighbourhood, person, nr_of_rooms, square_meters, interior_exterior, floor_nr, lift, garage])

    for apartment in property_details:
        writer.writerow(apartment)
file.close()
print(f'Number of rows scraped: {len(property_details)}')